In [ ]:
!pip install catboost

In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["Graph"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading Graph
9381571
loaded Graph


In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# submission_df.head()

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [ ]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [ ]:
del dataset_max
# del submission_df
# del submission_df_tr

In [ ]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [ ]:
gc.collect()

220

In [ ]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

0

In [ ]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [ ]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [ ]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

0.6230882352941176

In [ ]:
dataset.head(20)

,Session_Id,Item_ID,Score_Graph,score_LSTM,rank_LSTM,Max_Score_Graph,target
107,918383,55,0.005355,0.001339,47.0,91.79825,0
108,918383,189,0.020602,0.000000,0.0,91.79825,0
109,918383,260,0.003595,0.000000,0.0,91.79825,0
110,918383,357,0.003286,0.000000,0.0,91.79825,0
111,918383,379,0.004070,0.000850,73.0,91.79825,0
112,918383,421,0.013832,0.000775,80.0,91.79825,0
113,918383,506,0.003990,0.000000,0.0,91.79825,0
114,918383,531,0.006495,0.000982,65.0,91.79825,0
115,918383,548,0.044912,0.000748,87.0,91.79825,0
116,918383,609,0.004670,0.000000,0.0,91.79825,0


In [ ]:
dataset

,Session_Id,Item_ID,Score_Graph,score_LSTM,rank_LSTM,Max_Score_Graph,target
107,918383,55,0.005355,0.001339,47.0,91.798250,0
108,918383,189,0.020602,0.000000,0.0,91.798250,0
109,918383,260,0.003595,0.000000,0.0,91.798250,0
110,918383,357,0.003286,0.000000,0.0,91.798250,0
111,918383,379,0.004070,0.000850,73.0,91.798250,0
...,...,...,...,...,...,...,...
14333977,999999,1092,0.000000,0.001129,96.0,12.826448,0
14333978,999999,1513,0.000000,0.001127,97.0,12.826448,0
14333979,999999,827,0.000000,0.001115,98.0,12.826448,0
14333980,999999,3645,0.000000,0.001111,99.0,12.826448,0


In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [ ]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for All tradition models + LSTM models

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
from sklearn.model_selection import KFold
from catboost import CatBoostRanker, Pool
import os
import time

start_time = time.time()

k = 10

score = 0
kf = KFold(n_splits=k)
index = 0

mrr = 0
mrr_session_count = 0

for train_index, valid_index in kf.split(train):

    train_sessions = train["Session_Id"][train_index].tolist()
    valid_sessions = train["Session_Id"][valid_index].tolist()

    train_set = dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set = dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set = train_set.sort_values("Session_Id")
    valid_set = valid_set.sort_values("Session_Id")

    qids_train = train_set["Session_Id"]
    qids_valid = valid_set["Session_Id"]

    X_train = train_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_train = train_set["target"]

    X_valid = valid_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_valid = valid_set["target"]

    train_pool = Pool(X_train, y_train, group_id=qids_train)
    valid_pool = Pool(X_valid, y_valid, group_id=qids_valid)

    model = CatBoostRanker(
        iterations=2000,
        loss_function='YetiRank',
        od_type='Iter',
        od_wait=20,
        depth=6,
        l2_leaf_reg=3,
        learning_rate=0.05,
        random_seed=42,
    )
    ranker = model.fit(
        train_pool,
        eval_set=valid_pool,
        verbose=100,
        plot=False
    )
    ranker.save_model(f'./drive/MyDrive/Recommendation_system/rankers/catboost/graph_and_lstm/model_{index}.cbm')

    # Get best score from evals_result
    best_iter = ranker.get_best_iteration()
    best_score = ranker.get_evals_result()
    score += best_score['validation']['PFound'][best_iter]

    # Calculate MRR@100
    predictions_rank = train_set
    pred = (model.predict(predictions_rank))
    predictions_rank["score"]=np.mean(pred ,axis=0)

    gc.collect()

    predictions_rank=predictions_rank[["Session_Id", "Item_ID", "target", "score"]]
    predictions_rank=predictions_rank.groupby(['Session_Id']).agg(list).reset_index()
    predictions_rank["indices"]=predictions_rank["score"].apply(lambda x: get_indices(x))
    predictions_rank["predictions"]=predictions_rank[["Item_ID","indices"]].apply(get_items,axis=1)
    predictions_rank=predictions_rank[["Session_Id","predictions"]]
    predictions_rank["rank"]=predictions_rank.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

    original_purchase_df=train_set[["Session_Id", "Item_ID", "target", "score"]]
    original_purchase_df=original_purchase_df[original_purchase_df['target'] == 1]
    original_purchase_df=original_purchase_df.drop(["target", "score"], axis=1)
    original_purchase_df.rename(columns={'Item_ID': 'purchase_item_id'}, inplace=True)

    merged_df = pd.merge(predictions_rank, original_purchase_df, on='Session_Id', how='inner')

    gc.collect()

    for _, row in merged_df.iterrows():
        predictions = row["predictions"]
        purchase_item_id = row["purchase_item_id"]
        indices = np.where(predictions == purchase_item_id)[0]
        first_index = indices.min() + 1 if len(indices) > 0 else -1

        mrr += 1 / first_index if first_index > 0 else 0

    mrr_session_count += merged_df.shape[0]

    index += 1

end_time = time.time()

elapsed_time = end_time - start_time


0:	test: 0.2810554	best: 0.2810554 (0)	total: 4.92s	remaining: 2h 44m 1s
100:	test: 0.3706661	best: 0.3706661 (100)	total: 7m 53s	remaining: 2h 28m 16s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3716646679
bestIteration = 164

Shrink model to first 165 iterations.
0:	test: 0.3033593	best: 0.3033593 (0)	total: 4.96s	remaining: 2h 45m 6s
100:	test: 0.3836963	best: 0.3836963 (100)	total: 7m 56s	remaining: 2h 29m 13s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3844819379
bestIteration = 137

Shrink model to first 138 iterations.
0:	test: 0.2789557	best: 0.2789557 (0)	total: 5.07s	remaining: 2h 48m 53s
100:	test: 0.3624121	best: 0.3624121 (100)	total: 7m 55s	remaining: 2h 28m 54s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3638479629
bestIteration = 140

Shrink model to first 141 iterations.
0:	test: 0.3156945	best: 0.3156945 (0)	total: 4.9s	remaining: 2h 43m 18s
100:	test: 0.4059505	best: 0.4059505 (100)	total: 7m 53s

In [ ]:
mrr_final_performcance=(mrr / mrr_session_count) / k
print("overall MAP@100 = ", mrr_final_performcance)

overall MAP@100 =  0.0029678049231998457


In [ ]:
print("normalized score ", mrr_final_performcance*HITRATE)

normalized score  0.0018492043322937861


In [ ]:
pairwise_final_performcance=score/k
print("overall Pairwise = ", pairwise_final_performcance)

overall Pairwise =  0.3929912086674404


In [ ]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 10207.19841003418 seconds
